<a href="https://colab.research.google.com/github/Jomana-ElSaghier/Movie-Recommendation-System-Description/blob/main/Movie_Recommendation_System_Description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Install and Import Libraries**

In [1]:
!pip install scikit-learn pandas numpy

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score

#**Load the files**

In [3]:
from google.colab import files
uploaded = files.upload()

Saving u.data to u (1).data
Saving u.genre to u.genre
Saving u.info to u.info
Saving u.item to u.item


In [5]:
# Ratings data
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# Movies data
movies_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv(
    'u.item',
    sep='|',
    names=['movie_id', 'title'],
    usecols=[0,1],
    encoding='latin-1'
)


ratings.head()


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


#**Create User-Item Matrix**

In [6]:
user_item_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating')
user_item_matrix = user_item_matrix.fillna(0)
user_item_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#**Compute User Similarity**

In [7]:
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
user_similarity_df.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941


#**Recommend Movies**

In [8]:
def recommend_movies(user_id, num_recommendations=5):
    sim_scores = user_similarity_df[user_id]
    weighted_ratings = np.dot(sim_scores, user_item_matrix)
    sim_sums = sim_scores.sum()
    predicted_ratings = weighted_ratings / sim_sums
    predicted_ratings_series = pd.Series(predicted_ratings, index=user_item_matrix.columns)
    user_rated_movies = user_item_matrix.loc[user_id]
    predicted_ratings_series = predicted_ratings_series[user_rated_movies == 0]
    top_movies = predicted_ratings_series.sort_values(ascending=False).head(num_recommendations)

    return movies.set_index('movie_id').loc[top_movies.index]

In [9]:
recommendations = recommend_movies(user_id=1, num_recommendations=5)
print(recommendations)

                                           title
movie_id                                        
318                      Schindler's List (1993)
423            E.T. the Extra-Terrestrial (1982)
357       One Flew Over the Cuckoo's Nest (1975)
286                  English Patient, The (1996)
288                                Scream (1996)


#**Evaluate performance using precision at K**

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ratings, test_size=0.2, random_state=42)
train_matrix = train.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)
test_matrix = test.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

In [15]:
def recommend_movies(user_id, similarity_df, ratings_matrix, num_recommendations=5):
    # Get similarity scores for this user
    sim_scores = similarity_df[user_id]

    # Weighted sum of ratings
    weighted_ratings = np.dot(sim_scores, ratings_matrix)
    sim_sums = sim_scores.sum()
    predicted_ratings = weighted_ratings / (sim_sums if sim_sums != 0 else 1)

    # Convert to Series
    predicted_ratings_series = pd.Series(predicted_ratings, index=ratings_matrix.columns)

    # Remove already rated movies (from train set!)
    user_rated_movies = ratings_matrix.loc[user_id]
    predicted_ratings_series = predicted_ratings_series[user_rated_movies == 0]

    # Get top N
    top_movies = predicted_ratings_series.sort_values(ascending=False).head(num_recommendations)
    return movies.set_index('movie_id').loc[top_movies.index]


In [18]:
def precision_at_k(user_id, k=5):
    if user_id not in train_matrix.index or user_id not in test_matrix.index:
        return None

    # Get top-K recommendations using train data only
    top_k = recommend_movies(user_id, train_similarity_df, train_matrix, num_recommendations=k).index

    # Check relevance in test set
    user_test_ratings = test_matrix.loc[user_id]
    hits = 0
    for movie_id in top_k:
        if movie_id in user_test_ratings.index and user_test_ratings[movie_id] >= 4:
            hits += 1

    return hits / k

print("Precision@5 for user 1:", precision_at_k(1, k=5))


Precision@5 for user 1: 0.8
